## GAN

### Imports

In [1]:
import os
from tqdm.auto import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
from vr_recognition.data import Data
from vr_recognition.hardware import Hardware
from vr_recognition.model_utils import ModelUtils
from vr_recognition.vr_gesture_recognizer import VRGestureRecognizer
from vr_recognition.hardware import Hardware

### Data Loading

In [3]:
base_dir: str = os.path.join(os.getcwd(), 'simulated_data_v2')

tuple_data: tuple[list[pd.DataFrame], np.ndarray, np.ndarray] = Data.load_data(base_dir)

# Unpack data
pd_data: list[pd.DataFrame] = tuple_data[0]
labels: np.ndarray = tuple_data[1]
classes: np.ndarray = tuple_data[2]

0 VRAI HORIZONTAL
1 FAUX HORIZONTAL
2 FAUX VERTICAL
3 VRAI VERTICAL
4 NON VERTICAL
5 NON HORIZONTAL
6 OUI VERTICAL
7 OUI HORIZONTAL


In [4]:
data: np.ndarray = Data.convert_to_numpy(pd_data)

### Encoder

In [5]:
class Encoder(nn.Module):
    """
    Class defining the encoder architecture of an encoder.
    """
    def __init__(self, hidden_dim: int = 2):
        super(Encoder, self).__init__()
        self.conv1 = nn.Conv2d(1, hidden_dim, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(hidden_dim, hidden_dim, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(hidden_dim, hidden_dim * 2, kernel_size=3, stride=2, padding=1)
        self.conv4 = nn.Conv2d(hidden_dim * 2, hidden_dim * 2, kernel_size=3, stride=2, padding=1)

        # Activation functions
        self.relu = nn.LeakyReLU(0.1)

        # Dropout layer(s)
        self.dropout = nn.Dropout(p=0.3)
        
        # Pooling layer
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass of the model. This method is called when performing inference, training or evaluation.
        Params:
            x (torch.Tensor): Input data
        Returns:
            torch.Tensor: Output data.
                          The shape of the output tensor depends on the number of classes that characterizes the model.
        """
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        encoded = self.relu(self.conv4(x))

        encoded = self.dropout(encoded)
        return encoded

### Trim

In [6]:
class Trim(nn.Module):
    """
    Class that defines a trimming layer.
    """
    def __init__(self, size1: int, size2: int):
        super(Trim, self).__init__()
        self.size1 = size1
        self.size2 = size2
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass of the layer. This method is called when performing inference, training or evaluation.
        Params:
            x (torch.Tensor): Input data
        Returns:
            torch.Tensor: Trimmed data.
        """
        return x[:, :, :self.size1, :self.size2]

### Decoder

In [7]:
class Decoder(nn.Module):
    """
    Class defining the decoder architecture of the VAE.
    """
    def __init__(self, hidden_dim: int = 2, img_shape: tuple[int, int] = (72, 114)):
        super(Decoder, self).__init__()

        self.deconv1 = nn.ConvTranspose2d(hidden_dim * 2, hidden_dim * 2, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.deconv2 = nn.ConvTranspose2d(hidden_dim * 2, hidden_dim, kernel_size=3, stride=2, padding=1, output_padding=0)
        self.deconv3 = nn.ConvTranspose2d(hidden_dim, hidden_dim, kernel_size=3, stride=2, padding=1, output_padding=0)
        self.deconv4 = nn.ConvTranspose2d(hidden_dim, 1, kernel_size=3, stride=2, padding=1, output_padding=0)

        # Activation functions
        self.relu = nn.LeakyReLU(0.1)

        # Trimming
        self.trim = Trim(img_shape[0], img_shape[1])

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass of the decoder.
        Args:
            x (torch.Tensor): Input tensor.
        Returns:
            torch.Tensor: Decoded tensor representing the reconstructed data.
        """
        x = self.relu(self.deconv1(x))
        x = self.relu(self.deconv2(x))
        x = self.relu(self.deconv3(x))
        decoded = self.relu(self.deconv4(x))

        decoded = self.trim(decoded)
        
        return decoded

### Generative Adversarial Network

In [8]:
DEVICE = Hardware.device()

#### Generator

In [9]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()
        pass

#### Discriminator

In [10]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        pass

#### GAN

In [11]:
class GAN(nn.Module):
    """
    Class defining the GAN architecture.
    """
    pass